Agent: una entidad que percibe su entorno y las cosas que lo rodean
state: es una configuracion del agente en su entorno
initial state: es el punto de partida desde donde vamos a empezar a tratar de descubrir como ir de un estado a otro
actions: toma una entrada S donde S sera un estado que existe dentro de nuestro entorno y devolvera un conjunto de todas las acciones que se pueden realizar en ese estado
transition model: descripcion de que estado resultada despues de aplicar una accion
Result Toma una accion y un estado y retorna un nuevo estado
state space todos los estados que podemos tener de un estado inicial
grafo representa los estados y las flechas todos los posibles acciones que podemos tomar de una accion a otra
goal test una manera de determinar si un estado dado es el estado final
costo de la ruta path cost> es el valor asociado a una ruta el objetivo es encontrar la ruta que minimisa el costo
search problem
initial state 
actions
transition model
goal test
path cost function
optimal solution solution that have the lowest path cost among all solutions
node estructura de datos que vamos a utilizar para hacer un seguimiento de los estados el nodo padre la accion y el costo de la ruta

approach
Start with the frontier that contains the initia state
-repeat:
    if the fronties is empty, then no solution
    remove a node from the frontier
    if node contains goal state, return solution
    expand node, add resulting nodes to the frontier

revised approach
start with the frontair that contains the initial state
start with an empy explored set
repeat:
    if the fronties is empty, then no solution
    remove a node from the frontier
    if node contains goal state return solution
    add the node to the explored set
    expand node add resulting nodes to the frontier if they are not already in the frontier or the explored set

stack: last in first out data type
La primera busqueda en profundidad trata la frontera como una pila
busqueda en ampitud trata la frontera como una cola







In [1]:
import sys

In [2]:
class Node():
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent
        self.action = action


In [10]:
# depth first search
class StackFrontier():
    def __init__(self) -> None:
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)
    
    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)
    
    def empty(self):
        return len(self.frontier)==0
    
    def remove(self):
        if self.empty:
            raise Exception("No hay nodos en la frontera por eliminar") 
        else:
            node = self.frontier[-1]
            self.frontier = self.frontier[:-1]
            return node
        


In [4]:

class QueueFrontier(StackFrontier):

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node

In [12]:
class Maze():

    def __init__(self, filename):

        # Read file and set height and width of maze
        with open(filename) as f:
            contents = f.read()

        # Validate start and goal
        if contents.count("A") != 1:
            raise Exception("maze must have exactly one start point")
        if contents.count("B") != 1:
            raise Exception("maze must have exactly one goal")

        # Determine height and width of maze
        contents = contents.splitlines()
        self.height = len(contents)
        self.width = max(len(line) for line in contents)

        # Keep track of walls
        self.walls = []
        for i in range(self.height):
            row = []
            for j in range(self.width):
                try:
                    if contents[i][j] == "A":
                        self.start = (i, j)
                        row.append(False)
                    elif contents[i][j] == "B":
                        self.goal = (i, j)
                        row.append(False)
                    elif contents[i][j] == " ":
                        row.append(False)
                    else:
                        row.append(True)
                except IndexError:
                    row.append(False)
            self.walls.append(row)

        self.solution = None


    def print(self):
        solution = self.solution[1] if self.solution is not None else None
        print()
        for i, row in enumerate(self.walls):
            for j, col in enumerate(row):
                if col:
                    print("█", end="")
                elif (i, j) == self.start:
                    print("A", end="")
                elif (i, j) == self.goal:
                    print("B", end="")
                elif solution is not None and (i, j) in solution:
                    print("*", end="")
                else:
                    print(" ", end="")
            print()
        print()


    def neighbors(self, state):
        row, col = state
        candidates = [
            ("up", (row - 1, col)),
            ("down", (row + 1, col)),
            ("left", (row, col - 1)),
            ("right", (row, col + 1))
        ]

        result = []
        for action, (r, c) in candidates:
            if 0 <= r < self.height and 0 <= c < self.width and not self.walls[r][c]:
                result.append((action, (r, c)))
        return result


    def solve(self):
        """Finds a solution to maze, if one exists."""

        # Keep track of number of states explored
        self.num_explored = 0

        # Initialize frontier to just the starting position
        start = Node(state=self.start, parent=None, action=None)
        frontier = StackFrontier()
        frontier.add(start)

        # Initialize an empty explored set
        self.explored = set()

        # Keep looping until solution found
        while True:

            # If nothing left in frontier, then no path
            if frontier.empty():
                raise Exception("no solution")

            # Choose a node from the frontier
            node = frontier.remove()
            self.num_explored += 1

            # If node is the goal, then we have a solution
            if node.state == self.goal:
                actions = []
                cells = []
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                actions.reverse()
                cells.reverse()
                self.solution = (actions, cells)
                return

            # Mark node as explored
            self.explored.add(node.state)

            # Add neighbors to frontier
            for action, state in self.neighbors(node.state):
                if not frontier.contains_state(state) and state not in self.explored:
                    child = Node(state=state, parent=node, action=action)
                    frontier.add(child)


    def output_image(self, filename, show_solution=True, show_explored=False):
        from PIL import Image, ImageDraw
        cell_size = 50
        cell_border = 2

        # Create a blank canvas
        img = Image.new(
            "RGBA",
            (self.width * cell_size, self.height * cell_size),
            "black"
        )
        draw = ImageDraw.Draw(img)

        solution = self.solution[1] if self.solution is not None else None
        for i, row in enumerate(self.walls):
            for j, col in enumerate(row):

                # Walls
                if col:
                    fill = (40, 40, 40)

                # Start
                elif (i, j) == self.start:
                    fill = (255, 0, 0)

                # Goal
                elif (i, j) == self.goal:
                    fill = (0, 171, 28)

                # Solution
                elif solution is not None and show_solution and (i, j) in solution:
                    fill = (220, 235, 113)

                # Explored
                elif solution is not None and show_explored and (i, j) in self.explored:
                    fill = (212, 97, 85)

                # Empty cell
                else:
                    fill = (237, 240, 252)

                # Draw cell
                draw.rectangle(
                    ([(j * cell_size + cell_border, i * cell_size + cell_border),
                      ((j + 1) * cell_size - cell_border, (i + 1) * cell_size - cell_border)]),
                    fill=fill
                )

        img.save(filename)


In [ ]:
m = Maze('C:\\Users\\ahurt\\OneDrive\\Escritorio\\NotasDeProgramacion\\Introduction-to-IA\\search\\src\\maze1.txt')
print("Maze:")
m.print()
print("Solving...")
m.solve()
print("States Explored:", m.num_explored)
print("Solution:")
m.print()
m.output_image("maze.png", show_explored=True)

In [14]:
!python ./src/maze.py ./src/maze2.txt

Maze:

███                 █████████
█   ███████████████████   █ █
█ ████                █ █ █ █
█ ███████████████████ █ █ █ █
█                     █ █ █ █
█████████████████████ █ █ █ █
█   ██                █ █ █ █
█ █ ██ ███ ██ █████████ █ █ █
█ █    █   ██B█         █ █ █
█ █ ██ ████████████████ █ █ █
███ ██             ████ █ █ █
███ ██████████████ ██ █ █ █ █
███             ██    █ █ █ █
██████ ████████ ███████ █ █ █
██████ ████             █   █
A      ██████████████████████

Solving...
States Explored: 194
Solution:

███                 █████████
█   ███████████████████   █ █
█ ████                █ █ █ █
█ ███████████████████ █ █ █ █
█                     █ █ █ █
█████████████████████ █ █ █ █
█   ██********        █ █ █ █
█ █ ██*███ ██*█████████ █ █ █
█ █****█   ██B█         █ █ █
█ █*██ ████████████████ █ █ █
███*██             ████ █ █ █
███*██████████████ ██ █ █ █ █
███****         ██    █ █ █ █
██████*████████ ███████ █ █ █
██████*████             █   █
A******████████████

Busqueda des informada
utiliza la informacion disponible

busqueda informada
utiliza algun conocimiento del problema para encontrar una solucion mas eficiente

greedy best first search
busqueda codiciosa del mejor primero
utilizamos una funcion que va a determinar que nodo tomar

a* Search convina la busqueda codiciosa pero esta vez, agregaremos a nuestro nodo de desicion cuantoas pasos hemos tomado para llegar hasta ahi tomando el menor




Adversarial Search
minimax
Empezamos por asignar un valor a cada uno de los posibles resultados
gana 0 = -1
empate = 0
gana x = 1

entonces o sera el minimizador y X el maximizador
max(x) intenta maximizar el puntaje
Min(o) intena minimizar el puntaje

S0 estado inicial
Player el turno o que jugador debe jugar
Action movimientos legales
result el estado resuiltante despues de una accion
terminal revisa si ya se ha terminado el juego
Utility valor numerico final determina quien gana



dado un estado s
    MAX toma una accion a de Actions (que contiene todos los movimientos legales) que produzca el maximo valor de MIN(Result(s,a))
    MIN toma una accion a de Actions que produzca el minimo valor de MAX(Results(s,a))

function MAX-Value(state):
    if Terminal(state)
        return Utility(state)

    v = - Infinito
    para cada accion a de Actions(state s):
          v = max(v,Min-Value(Result(s,a)))  
    return v    

function Min-Value(state):
    if Terminal(state)
        return Utility(state)

    v =  Infinito
    para cada accion a de Actions(state s):
          v = min(v,Max-Value(Result(s,a)))  
    return v   


alpha betha prooning
Optimizaciones
 
 evaluation function evaluo que tna lejos debemos ir en el arbol para poder determinar que tan buena es la decicion  que estamos a punto de tomar